In [3]:
import numpy as np
import pandas as pd
!wget https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all1.csv
!wget https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all.csv

--2021-06-25 07:14:33--  https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299412 (292K) [text/plain]
Saving to: ‘all1.csv.1’

all1.csv.1          100%[===================>] 292.39K  --.-KB/s    in 0.07s   

2021-06-25 07:14:33 (4.00 MB/s) - ‘all1.csv.1’ saved [299412/299412]

--2021-06-25 07:14:33--  https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

## 데이터 수집

In [38]:
train = pd.read_csv('all1.csv', encoding=  'utf8', sep=',')
train['label']=(train['label']>=0).astype(int)

test = pd.read_csv('all.csv', encoding= 'utf8', sep=',', error_bad_lines=False)
test['label']=(test['label']>=0).astype(int)

data = pd.concat([test,train])
print('전체 데이터 개수 : ', (len(train) + len(test)))

print('train 데이터 개수 : ', len(train))
print('test 데이터 개수 : ', len(test))

print(len(data))

전체 데이터 개수 :  2763
train 데이터 개수 :  2086
test 데이터 개수 :  677
2763


## 데이터 전처리 

In [19]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 450kB 34.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [21]:
from konlpy.tag import Okt
okt = Okt()
def tokenizer(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return o

def processing(data):
  data['title'] = data['title'].str.replace('Hot-Line','')
  data['title'] = data['title'].str.replace('↑',' 상승 ')
  data['title'] = data['title'].str.replace('↓', ' 하락 ')

  data['title'] = data['title'].str.replace('[^a-zA-Z0-9ㄱ-힗 ]', '')
  
  tmp_o = []
  for sentence in data['title']:
    tmp_o.append(tokenizer(sentence))

  data['pos'] = tmp_o
  
  return data


test = processing(test)
train = processing(train)

In [22]:
test[:5]

,Unnamed: 0,Unnamed: 0.1,title,time,label,ChangeK,pos
0,0,4,종목명 to benefit from strong yen pay high dividends,2016-06-29 15:26:05,1,0.010402,"[종목, to, benefit, from, strong, yen, pay, high..."
1,1,5,종목명 외국인 VIP 증가에 실적 양호,2016-07-19 08:12:04,1,-0.002088,"[종목, 외국인, VIP, 증가, 실적, 양호]"
2,2,6,종목명 2분기 영업이익 314억원전년 동기比 448 상승,2016-08-05 17:29:04,1,0.006531,"[종목, 2분, 영업, 이익, 314억원, 동기, 448, 상승]"
3,3,7,마카오 카지노경기 26개월만에 회복파라다이스종목명 들썩,2016-09-02 16:04:04,0,0.010680,"[마카오, 카지노, 경기, 26, 개월, 회복, 파라다이스, 종목]"
4,4,8,종목명 중국인 VIP 정체내년 성장 동력 부재,2016-11-07 08:45:07,0,0.007851,"[종목, 중국인, VIP, 정체, 내년, 성장, 동력, 부재]"


## TF - IDF를 위한 Title 단어 형태소 분석

In [8]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
vectorizer=TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000)


X = vectorize.fit_transform(data['title'])

In [46]:
from sklearn.model_selection import train_test_split

RANDOM_SEED=42
TEST_SPLIT=0.2

y=np.array(data['label'])

X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [47]:
logistic_classifier = LogisticRegression(class_weight='balanced')
logistic_classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
accuracy = logistic_classifier.score(X_eval, y_eval)

print('Accuracy : {}'.format(accuracy))

Accuracy : 0.5768535262206148
